In [143]:
import requests
import json
import geopandas as gpd
import pandas as pd
import folium
import branca

In [290]:
def analyst():

    coords  = '43.6629,-79.3957'
    date = '06-08-2020'
    bins = [1800,3600,5400,7200,9000,10800]
    cutoff = f'&cutoffSec={bins[0]}&cutoffSec={bins[1]}&cutoffSec={bins[2]}&cutoffSec={bins[3]}&cutoffSec={bins[4]}&cutoffSec={bins[5]}'
    tm = '08:%2030am'
    walkReluctance = 2
    walk_dist = 1600
    transfer_time=180
    mode = 'WALK,TRANSIT'
    params = f'?toPlace={coords}&fromPlace={coords}&arriveBy=TRUE&mode={mode}&date={date}&time={tm}&maxWalkDistance={walk_dist}&walkReluctance={walkReluctance}&minTransferTime={transfer_time}'
    url = f'http://localhost:8080/otp/routers/current/isochrone'+params+cutoff
    response = requests.get(url)

    return(response.json())

In [291]:
data = analyst()
with open('data.geojson', 'w') as outfile:
    json.dump(data, outfile)


In [292]:
isochrone = gpd.read_file('data.geojson')

In [293]:
df60 = gpd.overlay(isochrone[isochrone['time']==3600],isochrone[isochrone['time']==1800], how='difference')
df90 = gpd.overlay(isochrone[isochrone['time']==5400],isochrone[isochrone['time']==3600], how='difference')
df120 = gpd.overlay(isochrone[isochrone['time']==7200],isochrone[isochrone['time']==5400], how='difference')
df150 = gpd.overlay(isochrone[isochrone['time']==9000],isochrone[isochrone['time']==7200], how='difference')
df180 = gpd.overlay(isochrone[isochrone['time']==10800],isochrone[isochrone['time']==9000], how='difference')

In [294]:
iso_geo = pd.concat([isochrone[isochrone['time']==1800],df60, df90, df120, df150, df180])
iso_geo

,id,time,geometry
5,fid-2434c8d8_1729109f74e_-7ff4,1800,"MULTIPOLYGON (((-79.43350 43.65170, -79.43300 ..."
4,fid-2434c8d8_1729109f74e_-7ff3,3600,"MULTIPOLYGON (((-79.53490 43.69710, -79.53510 ..."
3,fid-2434c8d8_1729109f74e_-7ff2,5400,"MULTIPOLYGON (((-79.13230 43.77970, -79.13240 ..."
2,fid-2434c8d8_1729109f74e_-7ff1,7200,"MULTIPOLYGON (((-78.89120 43.87140, -78.89140 ..."
1,fid-2434c8d8_1729109f74e_-7ff0,9000,"MULTIPOLYGON (((-79.03930 43.89730, -79.03770 ..."
0,fid-2434c8d8_1729109f74e_-7fef,10800,"MULTIPOLYGON (((-78.83570 43.92190, -78.83600 ..."


In [295]:
iso_data = iso_geo[['id','time']]
iso_data['min'] = iso_data['time']/60
iso_data = iso_data.astype({'min': 'int'})
iso_data

,id,time,min
5,fid-2434c8d8_1729109f74e_-7ff4,1800,30
4,fid-2434c8d8_1729109f74e_-7ff3,3600,60
3,fid-2434c8d8_1729109f74e_-7ff2,5400,90
2,fid-2434c8d8_1729109f74e_-7ff1,7200,120
1,fid-2434c8d8_1729109f74e_-7ff0,9000,150
0,fid-2434c8d8_1729109f74e_-7fef,10800,180


In [328]:
m = folium.Map(location=[43.6629,-79.3957], zoom_start=9)

folium.Choropleth(
    geo_data=iso_geo,
    name='choropleth',
    data=iso_data,
    columns=['id', 'min'],
    key_on='feature.properties.id',
    fill_color='Blues',
    fill_opacity=0.6,
    line_opacity=0.2,
    bins=[30,60,90,120,150,190],
    legend_name='Travel Time by Transit to UofT (minutes)'
).add_to(m)


from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Transit Travel Time From UofT (Minutes)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#eff3ff;opacity:0.7;'></span>30 Minutes</li>
    <li><span style='background:#c6dbef;opacity:0.7;'></span>60 Minutes</li>
    <li><span style='background:#9ecae1;opacity:0.7;'></span>90 Minutes</li>
    <li><span style='background:#6baed6;opacity:0.7;'></span>120 Minutes</li>
    <li><span style='background:#3182bd;opacity:0.7;'></span>150 Minutes</li>
    <li><span style='background:#08519c;opacity:0.7;'></span>120 Minutes</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)



m